In [1]:
import ee, geetools

ee.Initialize()

In [2]:
image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20200101T100319_20200101T100321_T32TQM')
string = image.geetools.toString('this is the image date: {CLOUDY_PIXEL_PERCENTAGE}')
print(string.getInfo())


EEException: String.replace: Parameter 'replacement' is required.

In [4]:
image.propertyNames().getInfo()

['system:version',
 'system:id',
 'DATATAKE_IDENTIFIER',
 'AOT_RETRIEVAL_ACCURACY',
 'SPACECRAFT_NAME',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
 'CLOUD_SHADOW_PERCENTAGE',
 'MEAN_SOLAR_AZIMUTH_ANGLE',
 'system:footprint',
 'VEGETATION_PERCENTAGE',
 'SOLAR_IRRADIANCE_B12',
 'SOLAR_IRRADIANCE_B10',
 'SENSOR_QUALITY',
 'SOLAR_IRRADIANCE_B11',
 'GENERATION_TIME',
 'SOLAR_IRRADIANCE_B8A',
 'FORMAT_CORRECTNESS',
 'CLOUD_COVERAGE_ASSESSMENT',
 'THIN_CIRRUS_PERCENTAGE',
 'system:time_end',
 'WATER_VAPOUR_RETRIEVAL_ACCURACY',
 'system:time_start',
 'DATASTRIP_ID',
 'PROCESSING_BASELINE',
 'SENSING_ORBIT_NUMBER',
 'NODATA_PIXEL_PERCENTAGE',
 'SENSING_ORBIT_DIRECTION',
 'GENERAL_QUALITY',
 'GRANULE_ID',
 'REFLECTANCE_CONVERSION_CORRECTION',
 'MEDIUM_PROBA_CLOUDS_PERCENTAGE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8',
 'DATATAKE_TYPE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7',
 'MEAN_INCIDENCE_AZIMUTH_